In [17]:
import os
import numpy as np
import re

In [27]:
path_to_annotations = '../Smol_set/annotations_complete_eng/'
vocabulary = set()

def parse_annotation_file(path_to_annotation_file):
    # open in latin-1 encoding to avoid UnicodeDecodeError
    with open(path_to_annotation_file, 'r', encoding='latin-1') as f:
        # read the doc as a string
        doc = f.read()
        # replace \n with space
        doc = doc.replace('\n', ' ')
        # get parts between all <TITLE> and </TITLE> tags
        titles = re.findall(r'<TITLE>(.*?)</TITLE>', doc)
        # get parts between all <DESCRIPTION> and </DESCRIPTION> tags
        descriptions = re.findall(r'<DESCRIPTION>(.*?)</DESCRIPTION>', doc)
        # get parts between all <NOTES> and </NOTES> tags
        notes = re.findall(r'<NOTES>(.*?)</NOTES>', doc)
        # get parts between all <LOCATION> and </LOCATION> tags
        locations = re.findall(r'<LOCATION>(.*?)</LOCATION>', doc)

        # split into words and add to vocabulary
        for title in titles:
            for word in title.split():
                word = word.strip(' .,;:!?()[]\{\}\'\"')
                word = word.lower()
                vocabulary.add(word)
        for description in descriptions:
            for word in description.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                vocabulary.add(word)
        for note in notes:
            for word in note.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                vocabulary.add(word)
        for location in locations:
            for word in location.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                vocabulary.add(word)

        # print('titles: ', titles)
        # print('descriptions: ', descriptions)
        # print('notes: ', notes)
        # print('locations: ', locations)

    return titles, descriptions, notes, locations

for (root, dirs, files) in os.walk(path_to_annotations, topdown=True):
    print(root)
    # print(dirs)
    # print(files)
    for file in files:
        # parses the annotation file and adds the words to the vocabulary
        parse_annotation_file(os.path.join(root, file))

# print('vocabulary: ', vocabulary)
with open('vocabulary.txt', 'w') as f:
    for word in vocabulary:
        f.write(word + '\n')


../Smol_set/annotations_complete_eng/
../Smol_set/annotations_complete_eng/01
../Smol_set/annotations_complete_eng/05
../Smol_set/annotations_complete_eng/04
../Smol_set/annotations_complete_eng/00
../Smol_set/annotations_complete_eng/03
../Smol_set/annotations_complete_eng/02
